# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Autumn 2025** </center>
---
### <center> **Examples on Structured Streaming (Kafka)** </center>
---
**Profesor**: Pablo Camarillo Ramirez

# Install Kafka SERVER

Go to `kafka` folder and run:

`docker compose up -d`

# Create SparkSession
## Install Kafka connector

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Create a data stream from a Kafka topic

In [ ]:
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-spark-example") \
            .load()

# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")

# Send transformed data to the Sink
query_a = df_input.writeStream \
            .trigger(processingTime='1 second') \
            .outputMode("append") \
            .format("console") \
            .start()

In [ ]:
query_a.stop()
sc.stop()

## Add a Streaming listener

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pcamarillor.streaming_listener import MyListener

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (Kafka)") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:4.0.0") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext

# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

# Add custom listener
spark.streams.addListener(MyListener())

In [ ]:
# Create the remote connection
kafka_df = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "kafka:9093") \
            .option("subscribe", "kafka-spark-example") \
            .load()


# Transform binary data to string
df_input = kafka_df.selectExpr("CAST(value AS STRING)")

# Send transformed data to the Sink
query_b = df_input.writeStream \
            .trigger(processingTime='3 second') \
            .outputMode("append") \
            .format("console") \
            .start()


query_b.awaitTermination(25)

In [ ]:
query_b.stop()
sc.stop()